In [1]:
import scipy.io as sio
import numpy as np
import pickle
import mne
# %matplotlib notebook
%matplotlib inline
import os
from multiprocessing import Pool


In [4]:
#Define Classes
clas=[['4000.0', '40000.0'], ['5000.0', '50000.0']]
folder='one'
try:
    os.mkdir(folder)
except:
    print("probably exists")
dataFolder='allData/'

probably exists


In [5]:
files=[]
for file in os.listdir(dataFolder):
    if file.endswith(".set"):
        files.append(file)
print(files)

['VP27_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP22_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP35_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP36_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP26_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP28_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP10_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP11_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP12_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP13_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP25_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP37_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP23_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP18_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set', 'VP09_importPz_rere

In [6]:
offset=0.1 #Seconds before the event to ignore
sampleLen=5 #Seconds


In [7]:
li=[[1, 2], [3, 44]]
[3, 44] in li

True

In [8]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
  

In [11]:
def getData(name):
    
    raw=mne.io.read_raw_eeglab(dataFolder+name)
    sRate=raw.info['sfreq']

    (events,
     event_dict)=mne.events_from_annotations(raw)
    importantEvents=[]
    ke=list(event_dict.keys())
    found=[False]*len(clas)
    for i in range(len(clas)):

        for k in range(len(clas[i])):
            if clas[i][k] in ke:
                found[i]=True
        clas[i]=intersection(event_dict, clas[i])
    for i in range(len(clas)):
        marker=[]
        for k in range(len(clas[i])):
            for p in range(len(events)):
                if events[p][2]==event_dict[clas[i][k]]:
                    corrupted=False
                    #Check to see if error marker present in sample
                    end=events[p][0]-((sampleLen+offset)*sRate)
    #                 star=events[p][0]-offset
                    if end<0:
                        end=0
                    for j in range(p, -1, -1):
                        if events[j][0]>=end:
                            if events[j][2]==event_dict['-999.0']:
                                corrupted=True
                        else:
                            break
                    if not corrupted:    
                        marker.append(events[p][0])
        importantEvents.append(marker)
    data=[]
    for i in range(len(importantEvents)):
        feature=[]
        for k in range(len(importantEvents[i])):
            star=int(importantEvents[i][k]-((sampleLen+offset)*sRate))
            end=int(importantEvents[i][k]-(offset*sRate))
            out, times=raw[0:62, star:end]
            print(out.shape)
    #         print(k)
            feature.append(out.transpose())
        data.append(np.array(feature))
    data=np.array(data)
    if found == [True]*len(found):
        pickle.dump(data, open(folder+'/'+name, 'wb'))
    else:
        print('SKIPPING', name, clas)
    return data

In [12]:
getData(files[0])

Reading allData/VP27_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Used Annotations descriptions: ['-999.0', '1.0', '1000.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '12.0', '17.0', '2.0', '2000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '237.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '255.0', '3.0', '3000.0', '4.0', '4000.0', '5.0', '6.0', '7.0', '9.0', '99.0']
(62, 2560)
(62, 2560)
(62, 2560)
(62, 2560)
SKIPPING VP27_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set [['4000.0'], []]


<ipython-input-11-bfafab4e5509>:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw=mne.io.read_raw_eeglab(dataFolder+name)


array([array([[[-0.01323119, -0.00038069,  0.0117891 , ...,  0.00308059,
          0.00064319,  0.00552224],
        [-0.01323155, -0.00038208,  0.01178734, ...,  0.0030792 ,
          0.00067451,  0.00552082],
        [-0.01323087, -0.00038026,  0.01178806, ...,  0.00307857,
          0.00070151,  0.00552066],
        ...,
        [-0.01323227, -0.00035814,  0.01180074, ...,  0.00309382,
          0.00064404,  0.00551969],
        [-0.01323102, -0.00035673,  0.01180265, ...,  0.00309388,
          0.00065036,  0.00551963],
        [-0.01323193, -0.00035673,  0.01180237, ...,  0.00309292,
          0.00067189,  0.0055186 ]],

       [[-0.01289751, -0.00039562,  0.01173292, ...,  0.00302849,
          0.00066244,  0.00543643],
        [-0.0128929 , -0.00039461,  0.01173543, ...,  0.00303101,
          0.00069436,  0.00543969],
        [-0.01289169, -0.00039575,  0.01173801, ...,  0.00303334,
          0.0007224 ,  0.00544202],
        ...,
        [-0.01301987, -0.00041951,  0.01173943,

In [46]:
p = Pool(10)
out=p.map(getData, files)


Reading allData/VP12_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP25_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt


<ipython-input-42-38e409cdaf5f>:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw=mne.io.read_raw_eeglab('allData/'+name)


Reading allData/VP28_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Used Annotations descriptions: ['-999.0', '1.0', '1000.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '111.0', '12.0', '168.0', '17.0', '177.0', '185.0', '2.0', '2000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '234.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '255.0', '3.0', '4.0', '40000.0', '5.0', '50000.0', '6.0', '8.0', '9.0', '99.0']
Reading allData/VP11_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP36_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP10_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP27_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP22_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP35_importPz_rerefLM_noEOGs_crop_c

<ipython-input-42-38e409cdaf5f>:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw=mne.io.read_raw_eeglab('allData/'+name)


Reading allData/VP08_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Used Annotations descriptions: ['-999.0', '1.0', '1000.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '110.0', '111.0', '12.0', '152.0', '153.0', '154.0', '155.0', '156.0', '157.0', '158.0', '159.0', '160.0', '161.0', '162.0', '164.0', '165.0', '166.0', '167.0', '168.0', '169.0', '17.0', '170.0', '171.0', '174.0', '175.0', '180.0', '181.0', '185.0', '2.0', '2000.0', '20000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '234.0', '237.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '255.0', '3.0', '30000.0', '4.0', '4000.0', '40000.0', '5.0', '50000.0', '6.0', '9.0', '99.0']
Reading allData/VP26_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt
Reading allData/VP13_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.fdt


...

Used Annotations descriptions: ['-999.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '110.0', '111.0', '12.0', '154.0', '155.0', '159.0', '161.0', '162.0', '163.0', '164.0', '165.0', '166.0', '167.0', '168.0', '169.0', '17.0', '170.0', '171.0', '172.0', '173.0', '174.0', '175.0', '176.0', '177.0', '178.0', '180.0', '181.0', '185.0', '187.0', '188.0', '2.0', '2000.0', '20000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '234.0', '237.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '3.0', '30000.0', '4.0', '40000.0', '5.0', '50000.0', '7.0', '99.0']
SKIPPING VP33_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set [[], []]


<ipython-input-42-38e409cdaf5f>:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw=mne.io.read_raw_eeglab('allData/'+name)


Used Annotations descriptions: ['-999.0', '1.0', '1000.0', '10000.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '110.0', '111.0', '12.0', '153.0', '154.0', '155.0', '156.0', '157.0', '158.0', '159.0', '160.0', '161.0', '162.0', '163.0', '164.0', '165.0', '166.0', '167.0', '169.0', '17.0', '171.0', '172.0', '174.0', '176.0', '178.0', '179.0', '192.0', '2.0', '20000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '234.0', '237.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '3.0', '30000.0', '4.0', '4000.0', '40000.0', '5.0', '50000.0', '7.0', '9.0', '99.0']


<ipython-input-42-38e409cdaf5f>:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw=mne.io.read_raw_eeglab('allData/'+name)


Used Annotations descriptions: ['-999.0', '1.0', '1000.0', '10000.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '111.0', '12.0', '152.0', '154.0', '155.0', '156.0', '157.0', '158.0', '159.0', '160.0', '161.0', '162.0', '163.0', '164.0', '165.0', '166.0', '167.0', '168.0', '169.0', '17.0', '170.0', '171.0', '172.0', '173.0', '174.0', '175.0', '176.0', '177.0', '179.0', '181.0', '183.0', '187.0', '2.0', '2000.0', '20000.0', '201.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '232.0', '233.0', '234.0', '237.0', '238.0', '239.0', '243.0', '251.0', '252.0', '253.0', '255.0', '3.0', '3000.0', '30000.0', '4.0', '40000.0', '5.0', '50000.0', '6.0', '7.0', '8.0', '9.0', '99.0']
SKIPPING VP29_importPz_rerefLM_noEOGs_crop_cleanChans_rerefLM_modMarker.set [[], []]


In [64]:
master=[]
classes=len(clas)
for i in range(classes):
    chunk=[]
    for k in range(len(out)):
        for p in range(len(out[k][i])):
            chunk.append(out[k][i][p])
    master.append(chunk)
pickle.dump(master, open(folder+'/'+'generic', 'wb'))